# Module 10 Assignment: Web Scraping with Selenium

### Submitted by :

* Muhammad Bilal Jamil (mjamil@mail.yu.edu)

### Submitted to : Prof. Chiazza

In [113]:
#Importing useful libraries to work with APIs:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs as s3
import logging
import boto3
from botocore.exceptions import ClientError
import awscli
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import time
import io
import os

### Web scrapping Charities website:

In [114]:
# loading the chrome driver file:
driver = webdriver.Chrome()

# loading the charities website:
driver.get('https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=T&d-49653-p=2&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=&orgId=&num1=&state=none&regType=ALL&num2=')

#identify xpath of location to select element
inputElement = driver.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/table[2]/tbody")

#identify the table to scrape
table = driver.find_element_by_css_selector('table.Bordered')

#printing the table:
print(table)

<selenium.webdriver.remote.webelement.WebElement (session="a6c76d28a81663dbafa309b7c9d8d5b9", element="88293ce7-5a9b-4852-9a1d-477f3fffae81")>


<ipython-input-114-ccd58e457001>:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  inputElement = driver.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/table[2]/tbody")
<ipython-input-114-ccd58e457001>:11: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  table = driver.find_element_by_css_selector('table.Bordered')


In [115]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:502: UserWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead")


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
2,1 CLUB COLOMBIA CARIBE INTERNATIONAL,15-79-51,NO DATA AVAILABLE,NFP,QUEENS,NY
3,1 in 9 The Long Island Breast Cancer Action Co...,05-26-54,113195645,NFP,BALDWIN,NY
4,Academy of General Dentistry Foundation,43-07-85,237310583,NFP,ALBANY,NY
5,ALS ASSOCIATION UPSTATE NEW YORK CHAPTER,21-23-76,412129176,NFP,ELBRIDGE,NY
6,"American Agora Foundation, Inc.",40-62-78,204000236,NFP,NEW YORK,NY
7,AMERICAN CLASSICAL ORCHESTRA INC,15-89-17,061072470,NFP,NEW YORK,NY
8,"AOPA Foundation, Inc.",40-51-26,208817225,NFP,FREDERICK,MD
9,ARNOLD GOLD FOUNDATION,21-38-43,223052098,NFP,ENGLEWOOD CLIFFS,NJ


In [116]:
# we see the first row is a blank which does not give any data so we will drop it:
df = df.iloc[1:]
print(df)

                                    Organization Name  NY Reg #  \
1                                 04/11 10:17 PM test  47-13-95   
2                1 CLUB COLOMBIA CARIBE INTERNATIONAL  15-79-51   
3   1 in 9 The Long Island Breast Cancer Action Co...  05-26-54   
4             Academy of General Dentistry Foundation  43-07-85   
5            ALS ASSOCIATION UPSTATE NEW YORK CHAPTER  21-23-76   
6                     American Agora Foundation, Inc.  40-62-78   
7                    AMERICAN CLASSICAL ORCHESTRA INC  15-89-17   
8                               AOPA Foundation, Inc.  40-51-26   
9                              ARNOLD GOLD FOUNDATION  21-38-43   
10                Balanda Community Association, Inc.  42-90-86   
11                            BLESSING WAY FOUNDATION  20-76-19   
12  BOYS & GIRLS CLUB OF THE NORTHTOWNS FOUNDATION...  01-23-16   
13                   BRAND FOUNDATION OF NEW YORK INC  00-12-93   
14                                   Briar Foundation  42-66-3

We see that that first row is now removed.

In [56]:
# Creating a functions which will help me create the S3 bucket in my AWS S3 environment:

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [55]:
create_bucket('m10-assignment-charities-bucket')

True

In [117]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  dealerships.nys
  information-arch-mbj-assignment-8a
  m10-assignment-charities-bucket


In [135]:

# prepare csv file name   

filename= 'charities_bureau_scrape_mbj_ia_spring' #name of your file
datetime = time.strftime("_%Y:%m:%d:%H:%M:%S") #timestamp
filenames3 = "%s%s.csv"%(filename,datetime) #name of the filepath and csv file

print(filenames3)

charities_bureau_scrape_mbj_ia_spring_2022:03:24:17:53:24.csv


The above 'charities_bureau_scrape_mbj_ia_spring_2022:03:24:17:53:24.csv' name will be used for creating the actual csv on to the S3 bucket

In [130]:
import io
import os

# creating a new variable as s3_client
s3_client = boto3.client('s3')

# writing df to a csv file
with io.StringIO() as csv_file:
    df.to_csv(csv_file,index=False)
    
    response = s3_client.put_object(
        Bucket='m10-assignment-charities-bucket', Key="files/charities_bureau_scrape_mbj_ia_spring_2022:03:24:17:48:12.csv", Body=csv_file.getvalue()
    )
    
    # setting status message conditions
    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")
    
    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


In [131]:
# Viewing the response if its successful:

print(response)

{'ResponseMetadata': {'RequestId': 'TVE3W9BVEGV5VQ6G', 'HostId': 'xBclDWSIglVxA8wg28Rm9qcG7lhY4/zK7T5+pPKvt+hx3jc7zwNdGANEnKcMKsuVXsxjZxmoTLM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'xBclDWSIglVxA8wg28Rm9qcG7lhY4/zK7T5+pPKvt+hx3jc7zwNdGANEnKcMKsuVXsxjZxmoTLM=', 'x-amz-request-id': 'TVE3W9BVEGV5VQ6G', 'date': 'Thu, 24 Mar 2022 21:50:24 GMT', 'etag': '"26069b1424913e6f73521bbcaf0db88b"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'ETag': '"26069b1424913e6f73521bbcaf0db88b"'}


In [132]:
# Now we will read the file directly from the S3 bucket:
response = s3_client.get_object(
        Bucket='m10-assignment-charities-bucket', Key="files/df.csv")

status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    s3_backto_df = pd.read_csv(response.get("Body"))
    print(s3_backto_df)
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200
                                    Organization Name  NY Reg #  \
0                                 04/11 10:17 PM test  47-13-95   
1                1 CLUB COLOMBIA CARIBE INTERNATIONAL  15-79-51   
2   1 in 9 The Long Island Breast Cancer Action Co...  05-26-54   
3             Academy of General Dentistry Foundation  43-07-85   
4            ALS ASSOCIATION UPSTATE NEW YORK CHAPTER  21-23-76   
5                     American Agora Foundation, Inc.  40-62-78   
6                    AMERICAN CLASSICAL ORCHESTRA INC  15-89-17   
7                               AOPA Foundation, Inc.  40-51-26   
8                              ARNOLD GOLD FOUNDATION  21-38-43   
9                 Balanda Community Association, Inc.  42-90-86   
10                            BLESSING WAY FOUNDATION  20-76-19   
11  BOYS & GIRLS CLUB OF THE NORTHTOWNS FOUNDATION...  01-23-16   
12                   BRAND FOUNDATION OF NEW YORK INC  00-12-93   
13            

In [133]:
# viewing the dataframe made directly from S3 bucket:
s3_backto_df

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
1,1 CLUB COLOMBIA CARIBE INTERNATIONAL,15-79-51,NO DATA AVAILABLE,NFP,QUEENS,NY
2,1 in 9 The Long Island Breast Cancer Action Co...,05-26-54,113195645,NFP,BALDWIN,NY
3,Academy of General Dentistry Foundation,43-07-85,237310583,NFP,ALBANY,NY
4,ALS ASSOCIATION UPSTATE NEW YORK CHAPTER,21-23-76,412129176,NFP,ELBRIDGE,NY
5,"American Agora Foundation, Inc.",40-62-78,204000236,NFP,NEW YORK,NY
6,AMERICAN CLASSICAL ORCHESTRA INC,15-89-17,061072470,NFP,NEW YORK,NY
7,"AOPA Foundation, Inc.",40-51-26,208817225,NFP,FREDERICK,MD
8,ARNOLD GOLD FOUNDATION,21-38-43,223052098,NFP,ENGLEWOOD CLIFFS,NJ
9,"Balanda Community Association, Inc.",42-90-86,264154665,NFP,DENVER,CO


### References:

* https://xcorr.net/2020/04/02/scraping-webpages-with-developer-tools-and-selenium/
* https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.put_object
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#id259
* https://towardsdatascience.com/reading-and-writing-files-from-to-amazon-s3-with-pandas-ccaf90bfe86c
* https://github.com/soumilshah1995/Learn-AWS-with-Python-Boto-3/blob/master/S3%20AWS.ipynb
* https://www.youtube.com/watch?v=3oIOWJ-JxJk&t=123s
* https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a